In [11]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import heapq
from IPython.display import HTML

# Data structures for Fortune's algorithm
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Edge:
    def __init__(self, start, direction):
        self.start = start
        self.direction = direction
        self.end = None
        self.twin = None

class Event:
    def __init__(self, x, point=None, arc=None):
        self.x = x
        self.point = point
        self.arc = arc
        self.valid = True
    def __lt__(self, other):
        return self.x < other.x

class Arc:
    def __init__(self, site):
        self.site = site
        self.prev = None
        self.next = None
        self.event = None
        self.edge_left = None
        self.edge_right = None

class Voronoi:
    def __init__(self, points):
        self.points = points
        self.events = []
        self.arcs = None
        self.edges = []
        # initialize site events
        for p in points:
            heapq.heappush(self.events, Event(-p.x, point=p))

    def process(self, animate=False):
        if animate:
            # setup figure and initial plot
            self.fig, self.ax = plt.subplots()
            xs = [p.x for p in self.points]
            ys = [p.y for p in self.points]
            margin = 1.0
            self.ax.set_xlim(min(xs) - margin, max(xs) + margin)
            self.ax.set_ylim(min(ys) - margin, max(ys) + margin)
            self.sweep_line, = self.ax.plot([], [], 'r--', lw=2)
            self.beach_line, = self.ax.plot([], [], 'g-', lw=1)
            self.ax.scatter(xs, ys, c='k')
            anim = animation.FuncAnimation(
                self.fig,
                self._animate_step,
                frames=len(self.events),
                interval=200,
                blit=True,
                repeat=False
            )
            return anim
        else:
            # process all events without animation
            while self.events:
                event = heapq.heappop(self.events)
                if not event.valid:
                    continue
                x = -event.x
                if event.point:
                    self._handle_site(event.point, x)
                else:
                    self._handle_circle(event.arc)
            return self.edges

    def _animate_step(self, frame):
        # pop next valid event
        while self.events:
            event = heapq.heappop(self.events)
            if event.valid:
                break
        else:
            return []
        x = -event.x
        # update sweep line
        y0, y1 = self.ax.get_ylim()
        self.sweep_line.set_data([x, x], [y0, y1])
        # handle event
        if event.point:
            self._handle_site(event.point, x)
        else:
            self._handle_circle(event.arc)
        # compute and draw beach line
        xs, ys = self._compute_beachline(x)
        self.beach_line.set_data(xs, ys)
        return [self.sweep_line, self.beach_line]

    def _compute_beachline(self, x, resolution=300):
        xs = np.linspace(self.ax.get_xlim()[0], self.ax.get_xlim()[1], resolution)
        ys = []
        for xv in xs:
            arc = self.arcs
            best = -np.inf
            best_y = None
            while arc:
                yv = self._parabola_y(arc.site, Point(xv, 0), x)
                if yv > best:
                    best = yv
                    best_y = yv
                arc = arc.next
            ys.append(best_y)
        return xs, ys

    def _handle_site(self, p, x):
        # insert a new parabola arc for site event
        if self.arcs is None:
            self.arcs = Arc(p)
            return
        arc = self._find_arc_above(p, x)
        if arc.event:
            arc.event.valid = False
        start = Point(x, self._parabola_y(arc.site, p, x))
        left_edge = Edge(start, np.array([arc.site.y - p.y, p.x - arc.site.x]))
        right_edge = Edge(start, np.array([p.y - arc.site.y, arc.site.x - p.x]))
        left_edge.twin = right_edge
        right_edge.twin = left_edge
        arc.edge_right = left_edge
        new_arc = Arc(p)
        new_arc.edge_left = right_edge
        # insert into linked list
        new_arc.prev = arc
        new_arc.next = arc.next
        if arc.next:
            arc.next.prev = new_arc
        arc.next = new_arc
        # record edges
        self.edges.extend([left_edge, right_edge])
        # check potential circle events
        self._check_circle(arc)
        self._check_circle(new_arc)
        if new_arc.next:
            self._check_circle(new_arc.next)

    def _handle_circle(self, arc):
        # remove arc from beachline
        if arc.prev:
            arc.prev.next = arc.next
        if arc.next:
            arc.next.prev = arc.prev
        # get circle center and intersection point
        ux, uy, r = arc.event.center
        pt = Point(ux, uy)
        # finalize disappearing arc's edges if they exist
        if arc.edge_left is not None:
            arc.edge_left.end = pt
        if arc.edge_right is not None:
            arc.edge_right.end = pt
        # create new bisector edge between neighbors
        if arc.prev and arc.next:
            new_edge = Edge(pt, np.array([
                arc.next.site.y - arc.prev.site.y,
                arc.prev.site.x - arc.next.site.x
            ]))
            arc.prev.edge_right = new_edge
            arc.next.edge_left = new_edge
            self.edges.append(new_edge)
        # invalidate any pending events on neighbors
        if arc.prev and arc.prev.event:
            arc.prev.event.valid = False
        if arc.next and arc.next.event:
            arc.next.event.valid = False
        # check new potential circles
        if arc.prev:
            self._check_circle(arc.prev)
        if arc.next:
            self._check_circle(arc.next)

    def _find_arc_above(self, p, x):
        arc = self.arcs
        while arc:
            if arc.next and self._parabola_y(arc.next.site, p, x) > self._parabola_y(arc.site, p, x):
                arc = arc.next
            else:
                break
        return arc

    def _parabola_y(self, site, p, x):
        # compute y-coordinate of parabola defined by site and directrix x
        if site.x == x:
            return site.y
        dp = 2 * (site.x - x)
        a = 1 / dp
        b = -2 * site.y / dp
        c = (site.y**2 + site.x**2 - x**2) / dp
        return a * p.x**2 + b * p.x + c

    def _check_circle(self, arc):
        # look for a circle event at triple (arc.prev, arc, arc.next)
        if not arc or not arc.prev or not arc.next:
            return
        A, B, C = arc.prev.site, arc.site, arc.next.site
        # must be a counter-clockwise turn
        if (B.x - A.x) * (C.y - A.y) >= (B.y - A.y) * (C.x - A.x):
            return
        # compute circumcircle center
        ax, ay = A.x, A.y
        bx, by = B.x, B.y
        cx, cy = C.x, C.y
        d = 2 * (ax*(by - cy) + bx*(cy - ay) + cx*(ay - by))
        if abs(d) < 1e-6:
            return
        ux = ((ax*ax+ay*ay)*(by-cy) + (bx*bx+by*by)*(cy-ay) + (cx*cx+cy*cy)*(ay-by)) / d
        uy = ((ax*ax+ay*ay)*(cx-bx) + (bx*bx+by*by)*(ax-cx) + (cx*cx+cy*cy)*(bx-ax)) / d
        r = np.hypot(A.x - ux, A.y - uy)
        ex = ux + r
        # must be ahead of sweep line
        if ex <= B.x:
            return
        e = Event(-ex, arc=arc)
        e.center = (ux, uy, r)
        arc.event = e
        heapq.heappush(self.events, e)


# Test cases
def test_simple():
    pts = [Point(0, 0), Point(1, 0), Point(0, 1)]
    v = Voronoi(pts)
    edges = v.process(animate=False)
    assert isinstance(edges, list), "Edges should be a list"
    assert len(edges) > 0, "Should generate at least one edge for 3 sites"
    print("test_simple passed")

if __name__ == '__main__':
    # run test
    test_simple()
    # run full animation in Colab
    np.random.seed(42)
    base = np.array([5, 5])
    points = [Point(*(base + np.random.randn(2))) for _ in range(10)]
    vor = Voronoi(points)
    anim = vor.process(animate=True)
    plt.close(vor.fig)
    display(HTML(anim.to_jshtml()))


test_simple passed
